In [38]:
!pip install --quiet open_atmos_jupyter_utils PyMPDATA

In [39]:
import os
os.environ['NUMBA_THREADING_LAYER'] = 'omp'

import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from open_atmos_jupyter_utils import show_plot, show_anim
from PyMPDATA import ScalarField, Solver, Stepper, VectorField, Options, boundary_conditions

In [40]:
g = 9.81
H_val = 200.0
Omega = 2 * np.pi / 86164
phi_deg = 80.0
phi_rad = np.deg2rad(phi_deg)
f_cor = 2 * Omega * np.sin(phi_rad)

c = np.sqrt(g * H_val)  # prędkość Kelvina

In [41]:
def plot_kelvin_initial_conditions(
    nx: int,
    ny: int,
    dx: float,
    dy: float,
    A: float = 0.1,
    H: float = H_val,
    filename: str = "kelvin_initial_conditions.pdf",
    show: bool = False,
):
    """
    Rysuje warunki początkowe dla fali Kelvina:
      - mapa 2D anomalii poziomu morza eta(x,y),
      - wykres 3D tej samej anomalii.

    Domyślnie:
      - zapisuje wykres do pliku `filename`,
      - NIE wyświetla go w notebooku (show=False).
    """

    # --- siatka fizyczna ---
    x = np.arange(nx) * dx        # [m], prostopadle do brzegu
    y = np.arange(ny) * dy        # [m], wzdłuż brzegu
    X, Y = np.meshgrid(x, y, indexing="ij")

    # --- warunek początkowy ---
    x_c = 0.85 * nx * dx          # środek paczki w oceanie
    Lx  = 0.05 * nx * dx          # szerokość w x
    y0  = 0.5  * ny * dy          # środek wzdłuż brzegu
    Ly  = 0.2  * ny * dy          # szerokość w y

    envelopex = np.exp(-((X - x_c) ** 2) / (2 * Lx ** 2))
    envelopey = np.exp(-((Y - y0) ** 2) / (2 * Ly ** 2))
    eta0 = A * envelopex * envelopey      # anomalia poziomu
    h0  = H + eta0                        # pełny poziom, jeśli by się przydał

    # współrzędne w km do ładnych osi
    Xkm = X / 1e3
    Ykm = Y / 1e3

    # --- figura z dwoma panelami: 2D + 3D ---
    fig = plt.figure(figsize=(12, 4))
    # jeszcze bliżej siebie: bardzo mały wspace
    gs = fig.add_gridspec(1, 2, wspace=0.02)

    # 1) mapa 2D anomalii
    ax1 = fig.add_subplot(gs[0, 0])
    pcm = ax1.pcolormesh(Ykm, Xkm, eta0, shading="auto")
    cb = fig.colorbar(pcm, ax=ax1, label=r"$\eta\ \mathrm{[m]}$", pad=0.02)
    ax1.set_xlabel(r"$y\ \mathrm{[km]}$", fontsize=14)
    ax1.set_ylabel(r"$x\ \mathrm{[km]}$", fontsize=14)

    # brak siatki na mapie 2D
    ax1.grid(False)

    # linia brzegu x=0
    ax1.axhline(0.0, color="red", linewidth=1.5)

    # 2) wykres 3D anomalii
    ax2 = fig.add_subplot(gs[0, 1], projection="3d")
    surf = ax2.plot_wireframe(
        Ykm, Xkm, eta0,
        color="blue", linewidth=0.5
    )
    ax2.set_xlabel(r"$y\ \mathrm{[km]}$", fontsize=14)
    ax2.set_ylabel(r"$x\ \mathrm{[km]}$", fontsize=14)
    # większy labelpad, żeby opis osi z był dobrze widoczny
    ax2.set_zlabel(r"$\eta\ \mathrm{[m]}$", fontsize=12, labelpad=10)

    # linia brzegu x=0 w 3D
    ax2.plot(
        [Ykm[0, 0], Ykm[0, -1]],
        [0.0, 0.0],
        [0.0, 0.0],
        color="red", linewidth=2.0,
    )

    # ciasny układ, ale z miejscem na suptitle
    fig.tight_layout(rect=[0, 0.0, 1, 0.93])

    # zapis do pliku (jeśli podany)
    if filename is not None:
        fig.savefig(filename, bbox_inches="tight")

    # kontrola wyświetlania
    if show:
        plt.show()
    else:
        plt.close(fig)

    return fig


In [42]:
nx0, ny0 = 128, 256
dx0, dy0 = 25e3, 25e3


fig = plot_kelvin_initial_conditions(nx0, ny0, dx0, dy0, A=0.1, H=H_val)



/tmp/ipython-input-1498600900.py:79: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  fig.tight_layout(rect=[0, 0.0, 1, 0.93])


In [43]:
class ShallowWaterEquationsIntegrator:
    def __init__(self, *, h_initial: np.ndarray, options: Options = None):
        """ initializes the solvers for a given initial condition of `h` assuming zero momenta at t=0 """

        options = options or Options(nonoscillatory=True, infinite_gauge=True)
        X, Y, grid = 0, 1, h_initial.shape
        stepper = Stepper(options=options, grid=grid)
        kwargs = {
            'boundary_conditions': (boundary_conditions.Constant(value=0), boundary_conditions.Periodic()),
            'halo': options.n_halo,
        }
        advectees = {
            "h": ScalarField(h_initial, **kwargs),
            "uh": ScalarField(np.zeros(grid), **kwargs),
            "vh": ScalarField(np.zeros(grid), **kwargs),
        }
        self.advector = VectorField((
                np.zeros((grid[X] + 1, grid[Y])),
                np.zeros((grid[X], grid[Y] + 1))
            ), **kwargs
        )
        self.solvers = { k: Solver(stepper, v, self.advector) for k, v in advectees.items() }

    def __getitem__(self, key):
        """ returns `key` advectee field of the current solver state """
        return self.solvers[key].advectee.get()

    def _apply_half_rhs(self, *, key, axis, g_times_dt_over_dxy):
        """ applies half of the source term in the given direction """
        self[key][:] -= .5 * g_times_dt_over_dxy * self['h'] * np.gradient(self['h'], axis=axis)

    def _update_courant_numbers(self, *, axis, key, mask, dt_over_dxy):
        """ computes the Courant number component from fluid column height and momenta fields """
        velocity = np.where(mask, np.nan, 0)
        momentum = self[key]
        np.divide(momentum, self['h'], where=mask, out=velocity)

        # using slices to ensure views (over copies)
        all = slice(None, None)
        all_but_last = slice(None, -1)
        all_but_first_and_last = slice(1, -1)

        velocity_at_cell_boundaries = velocity[(
            (all_but_last, all),
            (all, all_but_last),
        )[axis]] + np.diff(velocity, axis=axis) / 2
        courant_number = self.advector.get_component(axis)[(
            (all_but_first_and_last, all),
            (all, all_but_first_and_last)
        )[axis]]
        courant_number[:] = velocity_at_cell_boundaries * dt_over_dxy[axis]
        assert np.amax(np.abs(courant_number)) <= 1

    def __call__(self, *, nt: int,
                 g: float,
                 dt_over_dxy: tuple,
                 outfreq: int,
                 eps: float=1e-7,
                 f_cor: float=0.0,
                 sponge_width: int,
                 sigma_max: float,
                 dt: float): # dodano parametr Coriolisa
        """ integrates `nt` timesteps and returns a dictionary of solver states recorded every `outfreq` step[s] """
        output = {k: [] for k in self.solvers.keys()}
        f_times_dt = f_cor * dt  # assuming dt_over_dxy[0] == dt_over_dxy[1]
        for it in range(nt + 1):
            if it != 0:
                mask = self['h'] > eps
                for axis, key in enumerate(("uh", "vh")):
                    self._update_courant_numbers(axis=axis, key=key,
                                                mask=mask, dt_over_dxy=dt_over_dxy)

                self.solvers["h"].advance(n_steps=1)

                # --- momenta ---
                for axis, key in enumerate(("uh", "vh")):
                    # ciśnienie
                    self._apply_half_rhs(key=key, axis=axis,
                                        g_times_dt_over_dxy=g * dt_over_dxy[axis])

                # Coriolis – pierwszy półkrok (wspólny dla obu składowych)
                if f_cor != 0.0:
                    self._apply_coriolis_half_step(f_times_dt=f_times_dt, eps=eps)

                for axis, key in enumerate(("uh", "vh")):
                    self.solvers[key].advance(n_steps=1)

                # Coriolis – drugi półkrok
                if f_cor != 0.0:
                    self._apply_coriolis_half_step(f_times_dt=f_times_dt, eps=eps)

                # dokończenie ciśnienia
                for axis, key in enumerate(("uh", "vh")):
                    self._apply_half_rhs(key=key, axis=axis,
                                        g_times_dt_over_dxy=g * dt_over_dxy[axis])

                self['uh'][0, :] = 0.0   # brak przepływu przez brzeg x=0

                H = float(np.mean(self['h']))  # przybliżona głębokość tła

                self._apply_sponge_layer(dt=dt, H=H,
                            sponge_width=sponge_width,
                            sigma_max=sigma_max)



            if it % outfreq == 0:
                for key in self.solvers.keys():
                    output[key].append(self[key].copy())
        return output

    def _apply_coriolis_half_step(self, f_times_dt: float, eps: float):
        h = self['h']
        uh = self['uh']
        vh = self['vh']

        # przelicz pędy na prędkości tam gdzie jest woda
        u = np.zeros_like(h)
        v = np.zeros_like(h)
        mask = h > eps
        np.divide(uh, h, where=mask, out=u)
        np.divide(vh, h, where=mask, out=v)

        # człony: d(uh)/dt =  f * h * v
        #        d(vh)/dt = -f * h * u
        uh[:] += 0.5 * f_times_dt * h * v
        vh[:] -= 0.5 * f_times_dt * h * u
    def _apply_sponge_layer(self, dt: float, H: float,
                            sponge_width: int,
                            sigma_max: float):
        """
        Prosta warstwa wygaszająca przy dużym x:
        - w ostatnich 'sponge_width' kolumnach ściąga h -> H, uh -> 0, vh -> 0.
        - sigma(x) rośnie gładko od 0 do sigma_max.
        """
        h = self['h']
        uh = self['uh']
        vh = self['vh']

        nx, ny = h.shape
        x_start = max(0, nx - sponge_width)

        for i in range(x_start, nx):
            # i = x_start ... nx-1
            # normalizowany indeks od 0 do 1
            s = (i - x_start) / max(1, nx - 1 - x_start)
            # gładka funkcja od 0 do sigma_max (np. kwadratowa)
            sigma = sigma_max * s**2

            # czynnik tłumienia na jeden krok (ważne: sigma * dt << 1)
            factor = 1.0 - sigma * dt

            # tłumimy odchylenie od tła
            h[i, :] = H + (h[i, :] - H) * factor
            uh[i, :] *= factor
            vh[i, :] *= factor


In [44]:
def run_kelvin_sim(
    nx: int,
    ny: int,
    dx: float,
    dy: float,
    CFL: float = 0.4,
    nt: int = 1000,
    outfreq: int = 10,
    sponge_width: int = 10,
    sigma_max: float = 2e-3,
    A: float = 0.1,
):
    """
    Uruchamia jedną symulację Kelvina dla zadanej siatki i parametrów czasowych.

    Zwraca słownik z:
      - output: surowe dane z integratora (listy snapshotów 'h', 'uh', 'vh')
      - x, y, dx, dy
      - dt, times, outfreq
      - y0_index: indeks wzdłuż brzegu, gdzie była początkowo paczka
    """
    # krok czasowy z CFL
    dt = CFL * dx / c
    dt_over_dxy = (dt / dx, dt / dy)

    # siatka fizyczna
    x = np.arange(nx) * dx
    y = np.arange(ny) * dy
    X, Y = np.meshgrid(x, y, indexing="ij")

    # warunek początkowy
    x_c = 0.85 * nx * dx          # środek paczki w oceanie
    Lx = 0.05 * nx * dx           # szerokość w x
    y0 = 0.5 * ny * dy            # środek wzdłuż brzegu
    Ly = 0.2 * ny * dy            # szerokość w y

    envelopex = np.exp(-((X - x_c) ** 2) / (2 * Lx ** 2))
    envelopey = np.exp(-((Y - y0) ** 2) / (2 * Ly ** 2))
    eta0 = A * envelopex * envelopey

    h_initial = H_val + eta0

    # integrator MPDATA
    integrator = ShallowWaterEquationsIntegrator(h_initial=h_initial)

    output = integrator(
        nt=nt,
        g=g,
        dt_over_dxy=dt_over_dxy,
        outfreq=outfreq,
        f_cor=f_cor,
        sigma_max=sigma_max,
        sponge_width=sponge_width,
        dt=dt,
    )

    n_snaps = len(output["h"])
    times = np.arange(n_snaps) * outfreq * dt

    y0_index = int(y0 / dy)

    return {
        "output": output,
        "x": x,
        "y": y,
        "dx": dx,
        "dy": dy,
        "dt": dt,
        "times": times,
        "outfreq": outfreq,
        "y0_index": y0_index,
    }


In [68]:
def compute_kelvin_speed(
    h_series,
    H: float,
    dt: float,
    outfreq: int,
    dy: float,
    n_coastal: int = 4,
    direction: int = +1,
    amp_threshold: float = 1e-4,
    search_radius_km: float = 500.0,
    t_fit_fraction=(0.2, 0.8),
    c_guess = c,   # [m/s] – teoretyczna prędkość Kelvina
    dx: float | None = None,        # [m] – rozdzielczość w x (wzdłuż brzegu / poprzecznie)
    CFL: float | None = None,       # liczba CFL (opcjonalnie)
    L_R: float | None = None,       # promień Rossby'ego [m] (opcjonalnie)
    label: str = "",                # dodatkowy opis case'u
):
    """
    Szacuje prędkość fali Kelvina z Hovmöllera przy brzegu x=0.

    h_series : lista/array h[t, x, y]
    H        : głębokość tła
    dt       : krok czasowy schematu
    outfreq  : co ile kroków zapisywany jest stan
    dy       : rozdzielczość wzdłuż brzegu (y) [m]
    n_coastal: szerokość pasa przybrzeżnego (w komórkach)
    direction: +1 – fala idzie w stronę rosnących y,
               -1 – fala idzie w stronę malejących y
    amp_threshold   : minimalna amplituda
    search_radius_km: minimalny promień okna wyszukiwania [km]
    t_fit_fraction  : jaka część środkowa trajektorii jest używana do fitu
    c_guess         : teoretyczna prędkość Kelvina [m/s], jeśli podana,
                      to kolejne punkty są szukane w oknie y_pred ± c_guess*Δt
    dx, CFL, L_R    : używane tylko do opisu na wykresie (opcjonalnie)
    label           : dodatkowy opis case'u (np. "bazowa siatka")
    """
    h_arr = np.asarray(h_series)       # [nt, nx, ny]
    eta = h_arr - H
    n_snaps, nx, ny = eta.shape

    # --- Hovmöller przy brzegu: max po x w pasie przybrzeżnym ---
    stripe = eta[:, :n_coastal, :]          # [nt, n_coastal, ny]
    eta_edge = np.max(stripe, axis=1)       # [nt, ny]

    times = np.arange(n_snaps) * outfreq * dt       # [s]
    t_days = times / 86400.0
    y_coords = np.arange(ny) * dy                  # [m]
    y_km = y_coords / 1e3

    dt_snap = dt * outfreq                         # odstęp czasowy między snapshotami
    base_radius = search_radius_km * 1e3           # [m]

    # dodatnie części – grzbiety
    eta_pos = np.where(eta_edge > 0, eta_edge, 0.0)

    speed = np.nan
    y_pos = np.full(n_snaps, np.nan)

    # ----------------------------------------------------------
    # 1) WYBÓR POCZĄTKOWEGO GRZBIETU (SEED) W ODPOWIEDNIM ZAKRESIE y
    # ----------------------------------------------------------
    # dla direction=-1: szukamy w górnej 1/3 brzegu (duże y)
    # dla direction=+1: w dolnej 1/3 (małe y)
    if direction > 0:
        j_seed_min = 0
        j_seed_max = ny // 3
    else:
        j_seed_min = 2 * ny // 3
        j_seed_max = ny

    amp_best = 0.0
    seed_idx = None
    seed_j = None

    for it in range(n_snaps):
        A_y = eta_pos[it, j_seed_min:j_seed_max]
        a_max = A_y.max()
        if a_max > amp_threshold and a_max > amp_best:
            amp_best = a_max
            j_local = int(np.argmax(A_y))
            seed_idx = it
            seed_j = j_seed_min + j_local

    if seed_idx is None:
        # nie znaleziono sensownego grzbietu
        _plot_hovmoller(
            eta_edge, t_days, y_km, y_pos,
            times=None, fit_params=None,
            speed=np.nan, dt=dt, dy=dy,
            dx=dx, CFL=CFL, L_R=L_R, c_guess=c_guess, label=label,
        )
        return speed, times, y_pos

    y_pos[seed_idx] = y_coords[seed_j]

    # ----------------------------------------------------------
    # 2) ŚLEDZENIE GRZBIETU W PRZÓD I WSTECZ W CZASIE
    #    kolejne maksimum szukane w oknie wokół y_pred
    # ----------------------------------------------------------
    def track_step(it, y_ref, forward=True):
        """
        Zwraca nowe y dla indeksu czasu 'it' lub None,
        jeśli nic sensownego nie znaleziono.
        """
        A_y = eta_pos[it]
        if A_y.max() < amp_threshold:
            return None

        if c_guess is not None:
            # przewidywana pozycja grzbietu
            sign = direction if forward else -direction
            y_pred = y_ref + sign * c_guess * dt_snap
        else:
            y_pred = y_ref

        # przycinamy do domeny
        y_pred = np.clip(y_pred, y_coords[0], y_coords[-1])

        # promień wyszukiwania – max(base_radius, 2*c*Δt)
        r = base_radius
        if c_guess is not None:
            r = max(r, 2.0 * c_guess * dt_snap)

        mask = np.abs(y_coords - y_pred) <= r
        idx_cand = np.where(mask & (A_y > amp_threshold))[0]
        if idx_cand.size == 0:
            return None

        # wybierz lokalne maksimum najbliżej y_pred
        local_idx = []
        for j in idx_cand:
            if j == 0 or j == ny - 1:
                continue
            if A_y[j] >= A_y[j-1] and A_y[j] >= A_y[j+1]:
                local_idx.append(j)

        if local_idx:
            local_idx = np.array(local_idx)
            # najbliższe y_pred wśród lokalnych maksimów
            j_best = local_idx[np.argmin(np.abs(y_coords[local_idx] - y_pred))]
        else:
            # fallback: maksimum w oknie
            j_best = idx_cand[np.argmax(A_y[idx_cand])]

        return y_coords[j_best]

    # przód
    last_y = y_coords[seed_j]
    for it in range(seed_idx + 1, n_snaps):
        new_y = track_step(it, last_y, forward=True)
        if new_y is None:
            continue
        y_pos[it] = new_y
        last_y = new_y

    # wstecz
    last_y = y_coords[seed_j]
    for it in range(seed_idx - 1, -1, -1):
        new_y = track_step(it, last_y, forward=False)
        if new_y is None:
            continue
        y_pos[it] = new_y
        last_y = new_y

    # ----------------------------------------------------------
    # 3) DOPASOWANIE PROSTEJ DO TRAJEKTORII
    #    (tylko t > 1.4 dnia + filtr po amplitudzie)
    # ----------------------------------------------------------
    t_days_all = times / 86400.0

    # bierzemy tylko punkty, gdzie trajektoria istnieje ORAZ t > 1.4 dnia
    t_min_fit_days = 1.4
    valid_idx = np.where(
        np.isfinite(y_pos) & (t_days_all >= t_min_fit_days)
    )[0]

    if valid_idx.size >= 4:
        # czas i pozycja tylko w tych punktach
        t_valid = times[valid_idx]      # [s]
        y_valid = y_pos[valid_idx]      # [m]

        # amplituda na grzbiecie w tych punktach
        j_valid = np.clip((y_valid / dy).astype(int), 0, ny - 1)
        amp_valid = np.abs(eta_edge[valid_idx, j_valid])

        # wybór środkowej części trajektorii wg t_fit_fraction,
        # ALE już po odcięciu t < 1.4 dnia
        n_valid = valid_idx.size
        j0 = int(t_fit_fraction[0] * (n_valid - 1))
        j1 = int(t_fit_fraction[1] * (n_valid - 1)) + 1

        idx_win = valid_idx[j0:j1]
        t_win   = times[idx_win]
        y_win   = y_pos[idx_win]

        # amplituda w tym oknie
        j_win   = np.clip((y_win / dy).astype(int), 0, ny - 1)
        amp_win = np.abs(eta_edge[idx_win, j_win])

        # odrzucamy punkty o małej amplitudzie (np. < 40% maksimum w oknie)
        amp_cut = 0.4 * amp_win.max()
        good    = amp_win >= amp_cut

        t_fit = t_win[good]
        y_fit = y_win[good]

        if t_fit.size >= 2:
            # zwykłe dopasowanie prostej: y = a t + b
            a, b = np.polyfit(t_fit, y_fit, 1)
            speed = a
            fit_idx = idx_win[good]   # do rysowania czerwonej linii
            fit_params = (a, b, fit_idx)
        else:
            fit_params = None
    else:
        fit_params = None

    # ----------------------------------------------------------
    # 4) Wykres Hovmöllera z trajektorią i prostą
    # ----------------------------------------------------------
    _plot_hovmoller(
        eta_edge, t_days, y_km, y_pos,
        times=times, fit_params=fit_params,
        speed=speed, dt=dt, dy=dy,
        dx=dx, CFL=CFL, L_R=L_R, c_guess=c_guess, label=label,
    )

    return np.abs(speed), times, y_pos


def _plot_hovmoller(
    eta_edge, t_days, y_km, y_pos,
    times, fit_params,
    speed, dt, dy,
    dx: float | None = None,
    CFL: float | None = None,
    L_R: float | None = None,
    c_guess: float | None = None,
    label: str = "",
):
    T, Y = np.meshgrid(t_days, y_km, indexing="ij")
    fig, ax = plt.subplots(figsize=(8, 4))
    pcm = ax.pcolormesh(Y, T, eta_edge, shading="auto")
    cb = fig.colorbar(pcm, ax=ax)
    cb.set_label(r"$\eta\ \mathrm{[m]}$", fontsize=14)


    # --- tytuł z rozdzielczościami ---
    parts = []
    if dx is not None:
        parts.append(rf"$\Delta x={dx/1e3:.0f}\,\mathrm{{km}}$")
    if dy is not None:
        parts.append(rf"$\Delta y={dy/1e3:.0f}\,\mathrm{{km}}$")
    if dt is not None:
        parts.append(rf"$\Delta t={dt:.0f}\,\mathrm{{s}}$")

    title_base = "Propagacja brzegowej fali Kelvina (x=0)"
    if label:
        title_base += f" ({label})"
    if parts:
        title_base += " — " + ", ".join(parts)

    # ax.set_title(title_base)

    ax.set_xlabel("wzdłuż brzegu y [km]", fontsize=14)
    ax.set_ylabel("czas [dni]", fontsize = 14)

    # śledzony grzbiet
    ax.plot(y_pos/1e3, t_days, "w.", markersize=3, label="śledzony grzbiet")

    # dopasowana prosta
    if fit_params is not None and times is not None:
        a, b, fit_idx = fit_params
        t_line = np.linspace(times[fit_idx].min(), times[fit_idx].max(), 200)
        t_line_days = t_line / 86400.0
        y_line = a * t_line + b
        ax.plot(y_line/1e3, t_line_days, "r-", linewidth=2,
                label="dopasowana prosta")

    # --- pole tekstowe z parametrami ---
    lines = []
    if np.isfinite(speed):
        lines.append(
            rf"$|c_{{\rm num}}|={abs(speed):.3f}\,\mathrm{{m\,s^{{-1}}}}$"
        )
        if c_guess is not None and c_guess > 0:
            lines.append(
                rf"$|c_{{\rm num}}|/c={abs(speed)/c_guess:.3f}$"
            )
    if CFL is not None:
        lines.append(rf"\mathrm{{CFL}}={CFL:.2f}")
    if L_R is not None and dx is not None and dx > 0:
        lines.append(rf"$L_R/\Delta x={L_R/dx:.2f}$")
    if not lines:
        # minimalna informacja
        lines.append(rf"$\Delta t={dt:.0f}\,\mathrm{{s}}$")

    textstr = "\n".join(lines)
    ax.text(
        0.05, 0.25, textstr,
        transform=ax.transAxes,
        ha="left", va="top",
        bbox=dict(facecolor="white", alpha=0.8, edgecolor="black")
    )

    ax.legend(loc="upper right")
    plt.tight_layout()
    plt.savefig(rf"kelvin_hovmoller_fit_dt{dt:.0f}_dy{dy/1e3:.0f}.pdf")
    plt.close(fig)


In [47]:
def plot_kelvin_fields(sim: dict,
                       snapshot_index: int = 80,
                       label: str = "",
                       downsample: int = 4,
                       save_prefix: str | None = None,
                       make_animation: bool = False,
                       gif_file: str = "anim.gif",
                       make_hovmoller: bool = False,
                       przekroje: bool = False,
                       level_and_vector = False):
    out = sim["output"]
    x = sim["x"]
    y = sim["y"]
    dx = sim["dx"]
    dy = sim["dy"]
    times = sim["times"]
    y0_index = sim["y0_index"]

    L_R_km = L_R / 1e3

    # --- snapshot do statycznych rysunków ---
    h_snap = np.asarray(out["h"][snapshot_index])
    uh_snap = np.asarray(out["uh"][snapshot_index])
    vh_snap = np.asarray(out["vh"][snapshot_index])

    eta_snap = h_snap - H_val

    # prędkości
    u_snap = np.zeros_like(h_snap)
    v_snap = np.zeros_like(h_snap)
    mask_snap = h_snap > 1e-7
    u_snap[mask_snap] = uh_snap[mask_snap] / h_snap[mask_snap]
    v_snap[mask_snap] = vh_snap[mask_snap] / h_snap[mask_snap]

    Xkm, Ykm = np.meshgrid(x / 1e3, y / 1e3, indexing="ij")
    time_days = times[snapshot_index] / 86400.0

    if level_and_vector:
        # --- wybór zakresu w x: od brzegu do 2 * L_R ---
        max_x_km = 2.0 * L_R_km
        x_profile = Xkm[:, 0]
        ix_max = np.searchsorted(x_profile, max_x_km, side="right")

        Xkm_sub   = Xkm[:ix_max, :]
        Ykm_sub   = Ykm[:ix_max, :]
        eta_sub   = eta_snap[:ix_max, :]
        u_sub     = u_snap[:ix_max, :]
        v_sub     = v_snap[:ix_max, :]
        h_sub     = h_snap[:ix_max, :]

        # 1) mapa 2D anomalii poziomu morza (tylko 0–2 L_R)
        fig1, ax1 = plt.subplots(figsize=(8, 3))
        pcm = ax1.pcolormesh(Ykm_sub, Xkm_sub, eta_sub, shading="auto")
        cb = fig1.colorbar(pcm, ax=ax1)
        cb.set_label(r"$\eta\ \mathrm{[m]}$", fontsize=14)
        ax1.set_xlabel(r"$y\ \mathrm{[km]}$", fontsize=14)
        ax1.set_ylabel(r"$x\ \mathrm{[km]}$", fontsize=14)
        fig1.tight_layout()
        if save_prefix is not None:
            fig1.savefig(f"{save_prefix}_eta2D.pdf")
        plt.close(fig1)

        # 2) pole wektorowe prędkości (quiver)
        stride_x = max(1, h_sub.shape[0] // (downsample * 5))
        stride_y = max(1, h_sub.shape[1] // (downsample * 9))

        fig2, ax2 = plt.subplots(figsize=(8, 3))
        ax2.quiver(
            Ykm_sub[::stride_x, ::stride_y],
            Xkm_sub[::stride_x, ::stride_y],
            v_sub[::stride_x, ::stride_y],
            u_sub[::stride_x, ::stride_y],
            scale=None,
        )
        ax2.set_xlabel(r"$y\ \mathrm{[km]}$", fontsize=14)
        ax2.set_ylabel(r"$x\ \mathrm{[km]}$", fontsize=14)
        fig2.tight_layout()
        if save_prefix is not None:
            fig2.savefig(f"{save_prefix}_uv_quiver.pdf")
        plt.close(fig2)  # <-- zamykamy figurę

    if przekroje:
        # 3) przekrój poprzeczny (x) w okolicy y0
        x_km = x / 1e3
        fig3, ax3 = plt.subplots(figsize=(6, 3))
        ax3.plot(x_km, eta_snap[:, y0_index])
        ax3.set_xlabel(r"$x\ \mathrm{[km]}$")
        ax3.set_ylabel(r"$\eta\ \mathrm{[m]}$")
        ax3.set_title(rf"Przekrój poprzeczny przy $y \approx y_0$, $t={time_days:.1f}$ dni")
        fig3.tight_layout()
        if save_prefix is not None:
            fig3.savefig(f"{save_prefix}_section_x.pdf")
        plt.close(fig3)  # <-- zamiast plt.show()

        # 4) przekrój wzdłuż brzegu (y) przy brzegu (x=0)
        fig4, ax4 = plt.subplots(figsize=(6, 3))
        ax4.plot(y / 1e3, eta_snap[0, :])
        ax4.set_xlabel(r"$y\ \mathrm{[km]}$")
        ax4.set_ylabel(r"$\eta\ \mathrm{[m]}$")
        ax4.set_title(rf"Przekrój wzdłuż brzegu (x=0), $t={time_days:.2f}$ dni")
        fig4.tight_layout()
        if save_prefix is not None:
            fig4.savefig(f"{save_prefix}_section_y.pdf")
        plt.close(fig4)  # <-- zamiast plt.show()

    # --- 5) animacja 3D wireframe + GIF ---
    if make_animation:
        h_all = np.asarray(out["h"])  # [nt_snap, nx, ny]
        nt_snap, nx_snap, ny_snap = h_all.shape

        def plot(frame, *, zlim=(0.999 * H_val, 1.001 * H_val)):
            psi = h_all[frame]
            xi, yi = np.indices(psi.shape)
            fig, ax = plt.subplots(subplot_kw={"projection": "3d"},
                                   figsize=(12, 6))
            ax.plot_wireframe(xi + 0.5, yi + 0.5, psi,
                              color='blue', linewidth=.5)
            ax.set(
                zlim=zlim,
                proj_type='ortho',
                title=f"t / Δt = {frame}",
                zlabel=r"$\zeta$"
            )
            for axis in (ax.xaxis, ax.yaxis, ax.zaxis):
                axis.pane.fill = False
                axis.pane.set_edgecolor('black')
                axis.pane.set_alpha(1)
            for axis_name in ('x', 'y'):
                getattr(ax, f"set_{axis_name}label")(f"{axis_name} / Δ{axis_name}")

            ax.plot([0, 0], [0, ny_snap], [H_val, H_val],
                    color='red', linewidth=2)

            return fig  # <- dobrze, jeśli show_anim sam zamyka/nie wyświetla

        show_anim(plot, range(len(out["h"])), gif_file=gif_file)

    # --- 6) Hovmöller przy brzegu (x=0) ---
    if make_hovmoller:
        h_all = np.asarray(out["h"])   # [nt_snap, nx, ny]
        nt_snap, nx_snap, ny_snap = h_all.shape

        eta_edge = h_all[:, 0, :] - H_val   # x=0, funkcja (t, y)

        t_idx = np.arange(nt_snap)
        t_days = times / 86400.0

        Ykm = y / 1e3
        T, Ygrid = np.meshgrid(t_days, Ykm, indexing="ij")

        fig5, ax5 = plt.subplots(figsize=(8, 4))
        pcm = ax5.pcolormesh(Ygrid, T, eta_edge, shading="auto")
        cb = fig5.colorbar(pcm, ax=ax5)
        cb.set_label(r"$\eta\ [\mathrm{m}]$", fontsize=14)

        ax5.set_xlabel(r"$y\ \mathrm{[km]}$", fontsize=14)
        ax5.set_ylabel(r"$\mathrm{czas\ [dni]}$", fontsize=14)

        ax5.text(
            0.02, 0.95, r"$x=0$",
            transform=ax5.transAxes,
            color="white",
            fontsize=14,
            ha="left", va="top",
            bbox=dict(
                boxstyle="round,pad=0.3",
                facecolor="none",
                edgecolor="white",
                linewidth=1.0,
            ),
        )

        fig5.tight_layout()
        if save_prefix is not None:
            fig5.savefig(f"{save_prefix}_hovmoller.pdf")
        plt.close(fig5)


In [48]:
# parametry bazowe z Twojego kodu
dx0 = 25e3
dy0 = 25e3
nx0 = 128
ny0 = 256
CFL0 = 0.4
nt0 = 1000
outfreq0 = 10

# całkowity czas symulacji dla konfiguracji bazowej
dt0 = CFL0 * dx0 / c
T_total = nt0 * dt0  # [s]

# promień Rossby'ego
L_R = c / f_cor


In [49]:
# np. bazowa siatka:
sim_base = run_kelvin_sim(nx0, ny0, dx0, dy0, CFL=CFL0, nt=nt0, outfreq=outfreq0)


In [50]:
plot_kelvin_fields(sim_base, save_prefix="kelvin", level_and_vector=True, make_hovmoller=True)


**Zależność wyników od zwiększenia rozdzielczości przestrzennej**

In [51]:
space_factors = [0.5, 1, 2]
space_results = []

Lx_dom = nx0 * dx0
Ly_dom = ny0 * dy0

for f in space_factors:
    nx = nx0 * f
    ny = ny0 * f
    dx = Lx_dom / nx
    dy = Ly_dom / ny

    # chcemy ten sam czas fizyczny T_total
    dt = CFL0 * dx / c
    nt = int(T_total / dt)

    sim = run_kelvin_sim(
        nx=nx,
        ny=ny,
        dx=dx,
        dy=dy,
        CFL=CFL0,
        nt=nt,
        outfreq=outfreq0,
    )

    speed, times_traj, y_traj = compute_kelvin_speed(
      sim["output"]["h"], H_val,
      sim["dt"], sim["outfreq"], sim["dy"],
      n_coastal=4,
      direction=-1,
      amp_threshold=2e-3,
      search_radius_km=500.0,
      t_fit_fraction=(0.2, 0.8),
      c_guess=c,
  )


    space_results.append({
        "factor_space": f,
        "nx": nx,
        "ny": ny,
        "dx": dx,
        "dy": dy,
        "L_R_over_dx": L_R / dx,
        "speed_num": speed,
        "speed_theory": c,
        "speed_ratio": speed / c,
        "sim": sim,
    })




**Zależność wyników od zwiększenia rozdzielczości czasowej**

In [69]:
time_factors = [0.5, 1, 2]
time_results = []

for ft in time_factors:
    CFL = CFL0 / ft
    dt = CFL * dx0 / c
    nt = int(T_total / dt)

    sim = run_kelvin_sim(
        nx=nx0,
        ny=ny0,
        dx=dx0,
        dy=dy0,
        CFL=CFL,
        nt=nt,
        outfreq=outfreq0,
    )

    speed, times_traj, y_traj = compute_kelvin_speed(
        sim["output"]["h"], H_val, sim["dt"], sim["outfreq"], sim["dy"]
    )
    width = compute_cross_shore_width(
        sim["output"]["h"], H_val, sim["dx"], sim["y0_index"]
    )

    time_results.append({
        "factor_time": ft,
        "dt": sim["dt"],
        "nt": nt,
        "speed_num": speed,
        "speed_theory": c,
        "speed_ratio": speed / c,
        "width_sigma": width,
        "sim": sim,
    })


In [82]:
def visualize_space_results(space_results, *, save_prefix: str | None = None,
                            rys_c: bool = True,
                            rys_L: bool = True):
    """
    Wizualizacja zależności wyników od rozdzielczości przestrzennej.

    space_results – lista słowników jak w Twoim kodzie:
      {
        "factor_space": f,
        "nx": nx,
        "ny": ny,
        "dx": dx,
        "dy": dy,
        "L_R_over_dx": L_R / dx,
        "speed_num": speed,
        "speed_theory": c,
        "speed_ratio": speed / c,
        "width_sigma": width,
        "sim": sim,
      }
    save_prefix – jeśli nie None, zapisze wykresy jako
      save_prefix + "_speed_vs_res.pdf"
      save_prefix + "_width_vs_res.pdf"
    """

    # --- zrzut danych do tablic ---
    L_over_dx   = np.array([r["L_R_over_dx"] for r in space_results])
    speed_ratio = np.array([r["speed_ratio"]   for r in space_results])
    factors     = np.array([r["factor_space"]  for r in space_results])

    # sortowanie po L_R/dx (na wszelki wypadek)
    idx = np.argsort(L_over_dx)
    L_over_dx   = L_over_dx[idx]
    speed_ratio = speed_ratio[idx]
    factors     = factors[idx]


    if rys_c:
    # === 1) c_num / c vs L_R / Δx ===
      fig1, ax1 = plt.subplots(figsize=(6, 4))
      ax1.plot(L_over_dx, speed_ratio, "o-", label=r"obliczenia numeryczne")
      ax1.axhline(1.0, color="gray", linestyle="--",
                  label=r"wartość teoretyczna $c_\mathrm{num}/c = 1$")
      for Lval, rval, f in zip(L_over_dx, speed_ratio, factors):
          ax1.annotate(f"", (Lval, rval),
                      textcoords="offset points", xytext=(5, 5), fontsize=8)

      ax1.set_xlabel(r"$L_R / \Delta x$", fontsize=16)
      ax1.set_ylabel(r"$c_\mathrm{num} / c$", fontsize=16)
      # ax1.set_title(r"Zależność prędkości propagacji od rozdzielczości siatki")
      ax1.grid(True, linestyle=":", alpha=0.5)
      ax1.legend(fontsize=14)
      ax1.tick_params(axis='both', labelsize=12)
      fig1.tight_layout()

      if save_prefix is not None:
          fig1.savefig(f"{save_prefix}_speed_vs_res.pdf")
      plt.close(fig1)


In [83]:
def visualize_time_results(time_results, *, save_prefix: str | None = None,
                            rys_c: bool = True,
                            rys_L: bool = True):
    """
    Wizualizacja zależności wyników od rozdzielczości czasowej.

    time_results – lista słowników jak w Twoim kodzie:
      {
        "factor_time": ft,
        "dt": sim["dt"],
        "nt": nt,
        "speed_num": speed,
        "speed_theory": c,
        "speed_ratio": speed / c,
        "width_sigma": width,
        "sim": sim,
      }

    save_prefix – jeśli nie None, zapisze wykresy jako
      save_prefix + "_speed_vs_dt.pdf"
      save_prefix + "_width_vs_dt.pdf"
    """

    dt_arr       = np.array([r["dt"] for r in time_results])
    speed_ratio  = np.array([r["speed_ratio"]   for r in time_results])
    factors_time = np.array([r["factor_time"]   for r in time_results])

    # posortuj po dt (od największego do najmniejszego, albo odwrotnie – weźmy rosnąco)
    idx = np.argsort(dt_arr)
    dt_arr       = dt_arr[idx]
    speed_ratio  = speed_ratio[idx]
    factors_time = factors_time[idx]

    # === 1) c_num / c vs Δt ===
    if rys_c:
      fig1, ax1 = plt.subplots(figsize=(6, 4))
      ax1.plot(dt_arr, speed_ratio, "o-", label=r"obliczenia numeryczne")
      ax1.axhline(1.0, color="gray", linestyle="--",
                  label=r"wartość teoretyczna $c_\mathrm{num}/c = 1$")

      for dt_val, r_val, ft in zip(dt_arr, speed_ratio, factors_time):
          ax1.annotate(f"", (dt_val, r_val),
                      textcoords="offset points", xytext=(5, 5), fontsize=8)

      ax1.set_xlabel(r"$\Delta t\ \mathrm{[s]}$", fontsize=16)
      ax1.set_ylabel(r"$c_\mathrm{num} / c$", fontsize=16)
      # ax1.set_title(r"Zależność prędkości propagacji od kroku czasowego")
      ax1.grid(True, linestyle=":", alpha=0.5)
      ax1.legend(fontsize=14)
      ax1.tick_params(axis='both', labelsize=12)
      fig1.tight_layout()

      if save_prefix is not None:
          fig1.savefig(f"{save_prefix}_speed_vs_dt.pdf")
      plt.close(fig1)

In [84]:
visualize_space_results(space_results, save_prefix="kelvin_space_convergence", rys_L=False)
for r in space_results:
  plot_kelvin_fields(r["sim"],
                     snapshot_index=-1,
                     label=f"f={r['factor_space']}",
                     save_prefix=f"kelvin_space_convergence_f{r['factor_space']}")


**Zależność wyników od zwiększenia rozdzielczości czasowej**

In [85]:
visualize_time_results(time_results, save_prefix="kelvin_time_convergence", rys_L=False)
for r in time_results:
  plot_kelvin_fields(r["sim"],
                     snapshot_index=-1,
                     label=f"ft={r['factor_time']}",
                     save_prefix=f"kelvin_time_convergence_ft{r['factor_time']}")

\section{Cel projektu}

    Celem zrealizowanego projektu było numeryczne zbadanie propagacji brzegowej fali Kelvina w przybliżeniu równania płytkiej wody na prostokątnym obszarze basenu oceanicznego, a w szczególności odpowiedź na pytanie:
    \begin{quote}
        Czy korzystając z równań dynamiki płynu w przybliżeniu "płytkiej wody" można numerycznie odtworzyć brzegową falę Kelvina, oraz jak rozdzielczość siatki numerycznej wpływa na prędkość propagacji fali wzdłuż brzegu?
    \end{quote}

\section{Użyte równania}

    W modelu płytkiej wody rozpatrujemy równania
    
    $$
    \frac{\partial h}{\partial t}
    + \frac{\partial (hu)}{\partial x}
    + \frac{\partial (hv)}{\partial y} = 0,
    $$
    $$
    \frac{\partial (hu)}{\partial t}
    + \frac{\partial}{\partial x}
    \left(hu^2 + \frac{1}{2} g h^2 \right)
    + \frac{\partial (huv)}{\partial y}
    - f hv = 0,
    $$
    $$
    \frac{\partial (hv)}{\partial t}
    + \frac{\partial (huv)}{\partial x}
    + \frac{\partial}{\partial y}
    \left(hv^2 + \frac{1}{2} g h^2 \right)
    + f hu = 0,
    $$
    
    gdzie $h(x,y,t)$ jest wysokością słupa cieczy,
    $\mathbf{u} = (u,v)$ jest poziomym polem prędkości,
    $g$ – przyspieszeniem grawitacyjnym,
    a $f$ – parametrem Coriolisa.

\section{Warunki brzegowe i początkowe}
    
    W symulacji rozpatrywano prostokątny basen o wymiarach $L_x,\times L_y,$ z brzegiem wzdłuż $x = 0$. Wykorzystano następujące warunki brzegowe i początkowe.

    \begin{itemize}
        \item \textbf{warunki brzegowe}
        \begin{itemize}
            \item sztywny brzeg wzdłuż $x = 0$:
            \item brzegi wzdłuż osi $y$: periodyczne warunki brzegowe,
            \item brzeg wzdłuż osi $x$ "w głębi oceanu": wygaszanie fali, tzw. "gąbka".
        \end{itemize}
        \item \textbf{warunek początkowy} dla wysokości: gaussian w dużej odległości od brzegu.
    \end{itemize}

\section{Parametry numeryczne}
    
    W symulacji wykorzystano jednorodną siatkę numeryczną $\Delta x = \Delta y = 25$ km oraz krok czasowy $\Delta t$ dobrany tak, aby spełnić warunek CFL
    $$C = 0.4\quad \Longrightarrow\quad \Delta t = 225.76\text{ s}$$
    Przyjęto stałą głębokość tła $H = \text{200 m}$, szerokość geograficzną $\varphi = 80^\circ$. Dla analiz wpływu rozdzielczości siatki oraz kroku czasowego wykorzystano do symulacji po trzy wartości tych parametrów. Dla każdej z nich wyznaczono prędkość propagacji fali wzdłuż brzegu.


\section{Wyniki}
    
    W celu zilustrowania własności brzegowej fali Kelvina w przybliżeniu płytkiej wody na rysunkach \ref{fig:kelvin-eta2D}–\ref{fig:kelvin-hovmoller} przedstawiono zarówno strukturę przestrzenną pola wysokości swobodnej powierzchni, jak i odpowiadające jej pole prędkości oraz propagację sygnału w czasie wzdłuż linii brzegowej. Dla ustalonego czasu $t = 2.09$ dnia pokazano rozkład anomalii poziomu morza oraz skojarzone z nim pole prędkości, natomiast diagram Hovmöllera obrazuje ewolucję fali wzdłuż brzegu w całym rozważanym przedziale czasu. Na wszystkich rysunkach brzeg oceanu odpowiada współrzędnej $x=0$.
    
    \begin{figure}[H]
        \centering
        \includegraphics[width=.9\textwidth]{kelvin_eta2D.pdf}
        \caption{
            Pole anomalii poziomu morza $\eta(x,y)$ w modelu płytkiej wody
            w pobliżu pionowej linii brzegowej ($x=0$) dla czasu
            $t = 2.09$ dnia.
            Kolor reprezentuje odchylenie wysokości swobodnej powierzchni
            od stanu spoczynkowego, a wyraźne maksimum w pobliżu brzegu
            oraz stopniowe tłumienie sygnału w głąb oceanu są charakterystyczne
            dla brzegowej fali Kelvina propagującej się wzdłuż brzegu.}
        \label{fig:kelvin-eta2D}
    \end{figure}
    
    \begin{figure}[H]
        \centering
        \includegraphics[width=0.9\textwidth]{kelvin_uv_quiver.pdf}
        \caption{
            Pole wektorowe prędkości $(u,v)$ odpowiadające brzegowej fali
            Kelvina w tym samym momencie czasu $t = 2.09$ dnia, co na
            rysunku \ref{fig:kelvin-eta2D}.
            Strzałki wskazują kierunek i względną wartość prędkości, przy czym
            dominuje składowa równoległa do brzegu (wzdłuż osi $y$),
            natomiast składowa prostopadła do brzegu jest silnie tłumiona
            wraz ze wzrostem odległości $x$ od brzegu, zgodnie z teorią
            brzegowej fali Kelvina.}
        \label{fig:kelvin-uv-quiver}
    \end{figure}
    
    \begin{figure}[H]
        \centering
        \includegraphics[width=0.9\textwidth]{kelvin_hovmoller.pdf}
        \caption{
            Diagram Hovmöllera anomalii poziomu morza $\eta(t,y)$ przy brzegu
            ($x=0$). Na osi poziomej przedstawiono współrzędną wzdłuż brzegu
            $y$, na osi pionowej czas, a kolor oznacza wartość $\eta$.
            Pochyłe pasma dodatnich i ujemnych anomalii powierzchniowych
            obrazują propagację brzegowej fali Kelvina wzdłuż brzegu
            z prędkością zbliżoną do teoretycznej prędkości fazowej,
            natomiast prostokątna ramka z podpisem $x=0$ podkreśla, że
            pokazany jest wyłącznie sygnał przy samej linii brzegowej.}
        \label{fig:kelvin-hovmoller}
    \end{figure}
    Na poniższym rysunku przedstawiono wyniki symulacji dla dwukrotnie zwiększonego kroku czasowego i rozdzielczości przestrzennej.
    \begin{figure}[H]
        \centering
        \includegraphics[width=0.9\textwidth]{kelvin_hovmoller_fit_dt452_dy50.pdf}
        \caption{
            Diagram Hovmöllera anomalii poziomu morza $\eta(t,y)$ przy brzegu ($x=0$) dla $\Delta x = 50$ km i $\Delta t =452$ s. Białe punkty obrazują automatycznie śledzony grzbiet fali Kelvina, natomiast czerwona linia jest prostą dopasowaną do środkowej części trajektorii grzbietu. Z nachylenia tej prostej wyznaczono numeryczną prędkość fali $|c_{\mathrm{num}}| = 43.121\,\mathrm{m\cdot s^{-1}}$, co odpowiada relacji $|c_{\mathrm{num}}|/c = 0.974$ względem prędkości teoretycznej $c=\sqrt{gH}$.
        }
        \label{fig:kelvin-hovmoller-fit}
    \end{figure}

    Aby ocenić zbieżność numerycznego wyznaczania prędkości brzegowej fali Kelvina przeprowadzono osobno analizę wpływu kroku czasowego oraz rozdzielczości przestrzennej siatki numerycznej. Na rysunku \ref{fig:kelvin-time-convergence} przedstawiono zależność ilorazu $c_{\mathrm{num}}/c$ od kroku czasowego $\Delta t$ dla kilku wartości parametru $f_t$, natomiast rysunek \ref{fig:kelvin-space-convergence} pokazuje, jak ten sam iloraz zmienia się w funkcji bezwymiarowego parametru $L_R/\Delta x$ opisującego rozdzielczość promienia Rossby'ego na siatce. Oba wykresy pozwalają ocenić, w jakim zakresie krok czasowy i rozdzielczość przestrzenna gwarantują poprawne odtworzenie teoretycznej prędkości fali Kelvina.
    
    \begin{figure}[H]
        \centering
        \begin{subfigure}{0.48\textwidth}
            \centering
            \includegraphics[width=\linewidth]{kelvin_time_convergence_speed_vs_dt.pdf}
            \caption{Zależność $c_{\mathrm{num}}/c$ od kroku czasowego $\Delta t$.}
            \label{fig:kelvin-time-convergence}
        \end{subfigure}
        \hfill
        \begin{subfigure}{0.48\textwidth}
            \centering
            \includegraphics[width=\linewidth]{kelvin_space_convergence_speed_vs_res.pdf}
            \caption{Zależność $c_{\mathrm{num}}/c$ od $L_R/\Delta x$.}
            \label{fig:kelvin-space-convergence}
        \end{subfigure}
        \caption{
            Zbieżność numerycznej prędkości fali Kelvina $c_{\mathrm{num}}$
            względem wartości teoretycznej $c=\sqrt{gH}$ w funkcji
            parametrów numerycznych schematu: kroku czasowego $\Delta t$
            (panel~\subref{fig:kelvin-time-convergence}) oraz rozdzielczości
            przestrzennej $L_R/\Delta x$
            (panel~\subref{fig:kelvin-space-convergence}).
        }
        \label{fig:kelvin-convergence-both}
    \end{figure}


\section{Wnioski}

    Przeprowadzone obliczenia pokazały, że korzystając z równań dynamiki płynu w przybliżeniu płytkiej wody można numerycznie odtworzyć brzegową falę Kelvina w prostokątnym basenie oceanicznym. Uzyskane pola anomalii poziomu morza i prędkości mają wszystkie charakterystyczne cechy fali Kelvina: maksimum amplitudy zlokalizowane przy brzegu, wykładnicze tłumienie sygnału w głąb oceanu oraz przepływ wzdłuż linii brzegowej z silnie stłumioną składową prostopadłą. Diagramy Hovmöllera potwierdzają propagację sygnału w jednym kierunku wzdłuż brzegu z prędkością zbliżoną do teoretycznej prędkości fazowej. \\
    
    Analiza zbieżności pokazuje, że wpływ kroku czasowego na wartość $|c_{\text{num}}|/c$ jest niewielki, o ile zachowany jest warunek CFL, natomiast kluczowe znaczenie ma rozdzielczość przestrzenna: dla zbyt grubej siatki (małe $L_R/\Delta x$) prędkość fali jest niedoszacowana i dopiero dla wartości $L_R/\Delta x$ rzędu kilkunastu lub większych stosunek ten zbliża się do jedności. \\
    
    Podsumowując, odpowiedź na pytanie postawione w części wstępnej jest pozytywna: równania płytkiej wody z odpowiednio dobranymi warunkami brzegowymi pozwalają numerycznie odtworzyć brzegową falę Kelvina, a dokładność odtworzonej prędkości propagacji zależy przede wszystkim od rozdzielczości przestrzennej siatki numerycznej, przy zachowaniu rozsądnie małego kroku czasowego zgodnego z kryterium CFL.